In [1]:
from datetime import timedelta
import numpy as np
import pandas as pd
import json
import ta

# check https://technical-analysis-library-in-python.readthedocs.io/en/latest/ta.html#momentum-indicators for information on ta

In [2]:
# read in .json files
with open('stock_specific_info.json', 'r') as fp:
    stock_info_dict = json.load(fp)

with open('stock_historic_prices.json', 'r') as fp:
    stock_prices_dict = json.load(fp)

In [3]:
# get quarterly returns
quarter_returns = pd.read_csv('qtrlyReturns.csv')

In [4]:
# remove no quarter return stocks
drop_stocks = []

# add quarterly returns to stock_info_dict
for stock, quarter_info in stock_info_dict.items():
    stock_returns = quarter_returns[quarter_returns.Ticker == stock]
    try:
        stock_returns[['Q2 2019', 'Q3 2019', 'Q4 2019', 'Q1 2020', '5/21/2020']].astype('float', errors='raise')
    except:
        print('stock {} has bad qtr return values'.format(stock))
        drop_stocks.append(stock)
        continue
    stock_info_dict[stock]['2019_Q2'].update({ 'Return' : float(stock_returns['Q2 2019'].values[0] ), 'Return Next Quarter' : float(stock_returns['Q3 2019'])})
    stock_info_dict[stock]['2019_Q3'].update({ 'Return' : float(stock_returns['Q3 2019'].values[0] ), 'Return Next Quarter' : float(stock_returns['Q4 2019'])})
    stock_info_dict[stock]['2019_Q4'].update({ 'Return' : float(stock_returns['Q4 2019'].values[0] ), 'Return Next Quarter' : float(stock_returns['Q1 2020'])})
    stock_info_dict[stock]['2020_Q1'].update({ 'Return' : float(stock_returns['Q1 2020'].values[0] ), 'Return Next Quarter' : float(stock_returns['5/21/2020'])})

stock HEIA has bad qtr return values
stock JCP has bad qtr return values


In [5]:
# delete bad stocks
for stock in drop_stocks:
    del stock_info_dict[stock]
    del stock_prices_dict[stock]

In [95]:
quarters = ['2019_Q2', '2019_Q3', '2019_Q4', '2020_Q1']

# build some techinical indicators from price history for each stock
for stock in stock_prices_dict:  
    # convert to dataframe
    price_hist_df = pd.DataFrame.from_records(stock_prices_dict[stock]).T
    
    for qtr in quarters:
        # dictionary of indicators to add to stock info
        techinical_indicators = {}

        # momentum indicators
        aoi = ta.momentum.AwesomeOscillatorIndicator(high=price_hist_df.High, low=price_hist_df.Low).ao().loc[str(pd.to_datetime(stock_info_dict[stock][qtr]['Date']))]
        techinical_indicators['aoi'] = aoi

        kama = ta.momentum.KAMAIndicator(close=price_hist_df.Close).kama().loc[str(pd.to_datetime(stock_info_dict[stock][qtr]['Date']))]
        techinical_indicators['kama'] = kama

        roc = ta.momentum.ROCIndicator(close=price_hist_df.Close).roc().loc[str(pd.to_datetime(stock_info_dict[stock][qtr]['Date']))]
        techinical_indicators['roc'] = roc

        rsi = ta.momentum.RSIIndicator(close=price_hist_df.Close).rsi().loc[str(pd.to_datetime(stock_info_dict[stock][qtr]['Date']))]
        techinical_indicators['rsi'] = rsi

        tsi = ta.momentum.TSIIndicator(close=price_hist_df.Close).tsi().loc[str(pd.to_datetime(stock_info_dict[stock][qtr]['Date']))]
        techinical_indicators['tsi'] = tsi

        # volume indicators
        adi = ta.volume.AccDistIndexIndicator(high=price_hist_df.High, low=price_hist_df.Low, close=price_hist_df.Close, volume=price_hist_df.Volume).acc_dist_index().loc[str(pd.to_datetime(stock_info_dict[stock][qtr]['Date']))]
        techinical_indicators['adi'] = adi

        cmf = ta.volume.ChaikinMoneyFlowIndicator(high=price_hist_df.High, low=price_hist_df.Low, close=price_hist_df.Close, volume=price_hist_df.Volume).chaikin_money_flow().loc[str(pd.to_datetime(stock_info_dict[stock][qtr]['Date']))]
        techinical_indicators['cmf'] = cmf

        emi = ta.volume.EaseOfMovementIndicator(high=price_hist_df.High, low=price_hist_df.Low, volume=price_hist_df.Volume).ease_of_movement().loc[str(pd.to_datetime(stock_info_dict[stock][qtr]['Date']))]
        techinical_indicators['emi'] = emi
        
        
        # volatility indicators
        atr = ta.volatility.AverageTrueRange(high=price_hist_df.High, low=price_hist_df.Low, close=price_hist_df.Close).average_true_range().loc[str(pd.to_datetime(stock_info_dict[stock][qtr]['Date']))]
        techinical_indicators['atr'] = atr

        bband_h_indicator = ta.volatility.BollingerBands(close=price_hist_df.Close).bollinger_hband_indicator().loc[str(pd.to_datetime(stock_info_dict[stock][qtr]['Date']))]
        techinical_indicators['bband_h'] = bband_h_indicator

        bband_l_indicator = ta.volatility.BollingerBands(close=price_hist_df.Close).bollinger_lband_indicator().loc[str(pd.to_datetime(stock_info_dict[stock][qtr]['Date']))]
        techinical_indicators['bband_l'] = bband_l_indicator
        
        # trend indicators
        adx_pos = ta.trend.ADXIndicator(high=price_hist_df.High, low=price_hist_df.Low, close=price_hist_df.Close).adx_pos().loc[str(pd.to_datetime(stock_info_dict[stock][qtr]['Date']))]
        techinical_indicators['adx_pos'] = adx_pos

        adx_neg = ta.trend.ADXIndicator(high=price_hist_df.High, low=price_hist_df.Low, close=price_hist_df.Close).adx_neg().loc[str(pd.to_datetime(stock_info_dict[stock][qtr]['Date']))]
        techinical_indicators['adx_neg'] = adx_neg

        macd = ta.trend.MACD(close=price_hist_df.Close).macd_signal().loc[str(pd.to_datetime(stock_info_dict[stock][qtr]['Date']))]
        techinical_indicators['macd'] = macd

        # add to exisitng dictionary
        stock_info_dict[stock][qtr].update(techinical_indicators)


In [96]:
# turn stock info dictionary into dataframe
stock_info_df = pd.DataFrame.from_dict({(i,j): stock_info_dict[i][j]
                           for i in stock_info_dict.keys()
                           for j in stock_info_dict[i].keys()},
                       orient='index')

In [97]:
# combine index
stock_info_df = stock_info_df.set_index(stock_info_df.index.get_level_values(0) + ' ' + stock_info_df.index.get_level_values(1))

In [98]:
# convert to .csv
stock_info_df.to_csv('stock_complete_info.csv')
